# Setup

The below [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) extension ensures that if any locally imported python files change, the modules defined there are reloaded

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The below imports [`here`](https://pypi.org/project/pyprojroot/), which allows one to refer to the root directory of the project in a consistent manner across execution environments. It then adds `here()` (the root directory) to the system path to ensure that we can load python modules defined in the project.

In [8]:
#!pip install hereutil flask_sqlalchemy

from hereutil import here, add_to_sys_path
add_to_sys_path(here())

Having ensured that the root path of the project is in the system path, we can load common basis functions from [src/common_basis.py](/src/common_basis.py). The template assumes that functions useful for most work be defined in `common_basis.py`, whereas code useful for individual analyses is defined where needed.

Naturally, if more refined organisation of common code is needed, one is also free to define whichever other modules one wants.

The central object defined in `common_basis` is `con`, which is the [MariaDB](https://mariadb.com/) (MySQL) database connection (an [SQLAlchemy Connection](https://docs.sqlalchemy.org/en/14/core/connections.html)) through which both ready data is accessed, as well as new data stored for others to reuse. Below, you will see both how to use con to store data in the database, as well as how query it.

The details of the database connection are stored in [`db_params.yaml`](/db_params.yaml). The password is given separately. **DO NOT INCLUDE THE PASSWORD IN ANY CODE YOU COMMIT TO GITHUB**. If running this notebook, it will ask for the password the first time you run it, and then store it separately in your keyring. This requires a working keyring implementation on your system. Consult the [`keyring`](https://pypi.org/project/keyring/) package documentation if you have problems.  If you cannot get it to work, a second option is to create a `db_secret.yaml` file in the project root directory with `db_pass: [PASSWORD]` as the content. This file is already set to be ignored by Git so it wouldn't accidentally get included in a commit, but still, if you do this, **DON'T MAKE THE MISTAKE OF COMMITTING THE FILE TO GITHUB**.


In [9]:
from src.common_basis import *

eng, con = get_connection()

## Example of reading tweets from lynching_tweets_a table

The following code reads a random sample of tweets from lynching_tweets_a table with following specifications:

- **keyword**: each tweet returned by the query has the keyword as a substring
- **n** : upper limit of how many tweets are fetched
- **start_date**: fetches tweets where date_created_at is the start_date or later
- **end_date**: looks for tweets until the day end_date - 1 

In [10]:
keyword = "india"
n = 100
start_date = '2020-02-01'
end_date = '2023-02-05'

query = f"""
SELECT *
FROM lynching_tweets_a
WHERE MATCH(text) AGAINST('{keyword}' IN BOOLEAN MODE) 
AND created_at BETWEEN '{start_date}' AND '{end_date}'
ORDER BY RAND()
LIMIT {n};
"""

print("Query: ", query)

df = pd.read_sql(text(query), con)

Query:  
SELECT *
FROM lynching_tweets_a
WHERE MATCH(text) AGAINST('india' IN BOOLEAN MODE) 
AND created_at BETWEEN '2020-02-01' AND '2023-02-05'
ORDER BY RAND()
LIMIT 100;



NameError: name 'pd' is not defined

In [ ]:
# Show text column of the result 
df['text']